In [84]:
import math
import numpy as np
import pandas as pd

In [85]:
fdata = pd.read_csv('data_histories.csv', index_col="dp_folio")
fdata.head()

,id_sexo,Aedad,AAedad,Apuesto,id_gestud,AIMC,fuma,fuma_act,ejer_act,ejer1,...,locout5,locout10,locout20,locout30,rest_act,rest1,rest5,rest10,rest20,rest30
dp_folio,,,,,,,,,,,,,,,,,,,,,
1,F,51,6,Admin,CarTec,4,1,3,0,2,...,1,1,1,1,0,0,0,0,0,0
2,F,38,4,Sec,Bach,3,2,-1,0,0,...,0,0,0,0,1,3,0,0,0,0
3,F,34,3,Int,Sec,5,1,1,0,0,...,1,-1,-1,-1,0,0,0,-1,-1,-1
4,M,63,8,Jef,CarTec,4,2,-1,2,2,...,0,0,0,-1,0,0,0,0,0,0
5,M,42,4,EM,Sec,3,1,2,2,2,...,0,0,0,0,0,0,0,0,0,0


In [86]:
# Function to clusterize categories of a certain feature, and add the new clusterized feature as a new column
# Clusters should be an input of the form {cluster_A: {categories}, cluster_B: [categories]}
# Ex. obesity = {0:[1,2,3], 1:[4,5,6]}
def clusterizeDiscrete(feature, clusters, new_name, data):
    new_data = data.copy()
    original_list = fdata.loc[1:1080, feature]
    new_list = []
    for index in original_list.index:
        if original_list[index] == -1 or original_list[index] == "-1":
            new_list.append("N")
            continue
        for cluster in clusters:
            if original_list[index] in clusters[cluster]:
                new_list.append(cluster)
    new_data[new_name] = new_list
    return new_data

# Function to clusterize categories of a certain continous feature, and add the new clusterized feature as a
# new column
# Clusters shoud be an input of the form {cluster_A: {lambdaFunction1}, cluster_B: lambdaFunction2}
def clusterizeContinuous(feature, clusters, new_name, data):
    new_data = data.copy()
    original_list = fdata.loc[1:1080, feature]
    new_list = []
    for index in original_list.index:
        if original_list[index] == -1 or original_list[index] == "-1":
            new_list.append("N")
            continue
        for cluster in clusters:
            if eval(clusters[cluster])(original_list[index]):
                new_list.append(cluster)
                break
    new_data[new_name] = new_list
    return new_data
        

# Set of auxiliary high-order functions that will evaluate the conditions to binarize a history
def lessThan(num):
    return lambda n: n < num

def lessQThan(num):
    return lambda n: n <= num

def greaterThan(num):
    return lambda n: n > num

def greaterQThan(num):
    return lambda n: n >= num

def between(num1, num2):
    return lambda n: n <= num2 and n >= num1

In [87]:
# Clusterize between obese and non obese
obesity = {0:[1,2,3], 1:[4,5,6]}
ndata = clusterizeDiscrete("AIMC", obesity, "Obesity", fdata)

# Clusterize between degrees of study (higher degree vs. non higher degree)
degree = {0:["Prim", "Sec", "Bach", "CarTec", "Otro"], 1:["Lic", "Mast", "Doc", "PDoc"]}
ndata = clusterizeDiscrete("id_gestud", degree, "hdegree", ndata)

# Clusterize excercise features
ejer = {"B":"lessThan(2.5)", "A":"greaterQThan(2.5)"}
ndata = clusterizeContinuous("ejer_act", ejer, "ejer0B", ndata)
ndata = clusterizeContinuous("ejer1", ejer, "ejer1B", ndata)
ndata = clusterizeContinuous("ejer5", ejer, "ejer5B", ndata)
ndata = clusterizeContinuous("ejer10", ejer, "ejer10B", ndata)
ndata = clusterizeContinuous("ejer20", ejer, "ejer20B", ndata)
ndata = clusterizeContinuous("ejer30", ejer, "ejer30B", ndata)

# Clusterize 

In [126]:
#test = "clusterize('AIMC', obesity, 'Obesity', fdata)"
#eval(test)

#def lessThan(num):
#    return lambda n: n < num

#eval("lessThan(3)")(2)

In [88]:
ndata.head()

,id_sexo,Aedad,AAedad,Apuesto,id_gestud,AIMC,fuma,fuma_act,ejer_act,ejer1,...,rest20,rest30,Obesity,hdegree,ejer0B,ejer1B,ejer5B,ejer10B,ejer20B,ejer30B
dp_folio,,,,,,,,,,,,,,,,,,,,,
1,F,51,6,Admin,CarTec,4,1,3,0,2,...,0,0,1,0,B,B,B,B,B,A
2,F,38,4,Sec,Bach,3,2,-1,0,0,...,0,0,0,0,B,B,A,A,A,B
3,F,34,3,Int,Sec,5,1,1,0,0,...,-1,-1,1,0,B,B,B,A,N,N
4,M,63,8,Jef,CarTec,4,2,-1,2,2,...,0,0,1,0,B,B,A,A,A,A
5,M,42,4,EM,Sec,3,1,2,2,2,...,0,0,0,0,B,B,B,B,B,B


In [89]:
# Functions to count ocurrences for a category of a feature (NX)

# Count the number of instances inside the database whose feature X = category
# Ex: getNX('AIMC', 3, fdata)
def getNX(feature, category, data):
    count = 0
    for index in data.index:
        if data.loc[index][feature] == category:
            count = count + 1
    return count

# Count the number of instances inside the database whose feature X_1 = category_1 and feature X_2 = category_2
# Ex: getNCX('AIMC', 3, 'Obesidad', 1, fdata)
def getNCX(feature_1, category_1, feature_2, category_2, data):
    count = 0
    for index in data.index:
        if data.loc[index][feature_1] == category_1 and data.loc[index][feature_2] == category_2:
            count = count + 1
    return count

# Get a conditional probability P(F_1 = C_1 | F_2 = C_2)
def getCondProb(feature_1, category_1, feature_2, category_2, data):
    ncx = getNCX(feature_1, category_1, feature_2, category_2, data)
    nx = getNX(feature_2, category_2, data)
    if nx != 0:
        p = ncx / float(nx)
    else:
        p = 0
    #return {'P': p, 'nx': nx, 'ncx': ncx}
    return p

def getEpsilon(feature, category, classFeature, classCategory, data):
    n = len(data)
    nx = getNX(feature, category, data)
    nc = getNX(classFeature, classCategory, data)
    ncx = getNCX(feature, category, classFeature, classCategory, data)
    if n != 0 and nx != 0:
        pc = nc / float(n)
        pcx = ncx / float(nx)
        epsilon = nx * (pcx - pc) / math.sqrt(nx * pc * (1 - pc))
    else:
        epsilon = 0
    #print 'Epsilon :' + str(epsilon)
    return {'epsilon': epsilon, 'nx': nx, 'ncx': ncx, 'nc': nc}

In [90]:
getEpsilon("ejer1B", "B", "ejer5B", "A", ndata)

{'epsilon': -8.954266827130171, 'nc': 565, 'ncx': 209, 'nx': 608}

In [91]:
getCondProb("ejer1B", "B", "ejer5B", "A", ndata[ndata.Obesity==0])

0.3448275862068966

In [92]:
ndata[ndata.Obesity == 1]

,id_sexo,Aedad,AAedad,Apuesto,id_gestud,AIMC,fuma,fuma_act,ejer_act,ejer1,...,rest20,rest30,Obesity,hdegree,ejer0B,ejer1B,ejer5B,ejer10B,ejer20B,ejer30B
dp_folio,,,,,,,,,,,,,,,,,,,,,
1,F,51,6,Admin,CarTec,4,1,3,0,2,...,0,0,1,0,B,B,B,B,B,A
3,F,34,3,Int,Sec,5,1,1,0,0,...,-1,-1,1,0,B,B,B,A,N,N
4,M,63,8,Jef,CarTec,4,2,-1,2,2,...,0,0,1,0,B,B,A,A,A,A
13,F,41,4,Sec,CarTec,5,1,2,1,0,...,1,0,1,0,B,B,B,B,B,B
17,F,52,6,Sec,CarTec,4,2,-1,0,0,...,1,2,1,0,B,B,B,B,B,B
21,M,35,3,Acade,Mast,4,1,2,0,0,...,-1,-1,1,1,B,B,A,A,N,N
25,F,41,4,Vig,Sec,4,2,-1,0,1,...,0,0,1,0,B,B,B,B,B,B
36,F,59,8,Tec,Lic,4,1,2,4,4,...,0,1,1,1,A,A,A,A,B,A
40,M,44,5,Int,CarTec,6,1,1,0,0,...,0,0,1,0,B,B,B,A,A,A


In [108]:
# PENDING: Modularize this function, as it only works now with binary type variables
def transitionProbabilities(feature, data):
    years = ["30", "20", "10", "5", "1", "0"]
    transitionProbs = {}
    transitionProbs["labels"] = []
    transitionProbs["probs"] = []
    transitionProbs["pattern"] = []
    for i in range(len(years)):
        for j in range(i):
            feature_1 = feature + years[i] + "B"
            feature_2 = feature + years[j] + "B"
            f_1 = "a_" + years[i]
            f_2 = "a_" + years[j]
            label_1 = f_1 + ": A | " + f_2 + ": A "
            label_2 = f_1 + ": B | " + f_2 + ": A "
            label_3 = f_1 + ": A | " + f_2 + ": B "
            label_4 = f_1 + ": B | " + f_2 + ": B "
            prob_1 = getCondProb(feature_1, "A", feature_2, "A", data)
            prob_2 = getCondProb(feature_1, "B", feature_2, "A", data)
            prob_3 = getCondProb(feature_1, "A", feature_2, "B", data)
            prob_4 = getCondProb(feature_1, "B", feature_2, "B", data)
            transitionProbs["labels"].append(label_1)
            transitionProbs["labels"].append(label_2)
            transitionProbs["labels"].append(label_3)
            transitionProbs["labels"].append(label_4)
            transitionProbs["probs"].append(prob_1)
            transitionProbs["probs"].append(prob_2)
            transitionProbs["probs"].append(prob_3)
            transitionProbs["probs"].append(prob_4)
            transitionProbs["pattern"].append("A->A")
            transitionProbs["pattern"].append("A->B")
            transitionProbs["pattern"].append("B->A")
            transitionProbs["pattern"].append("B->B")
            #print label_1 + str(prob_1)
            #print label_2 + str(prob_2)
            #print label_3 + str(prob_3)
            #print label_4 + str(prob_4)
    return pd.DataFrame(data = transitionProbs)

# Function to explore subgroups? 

In [109]:
# Non Obese vs. Obese
# Get probabilities from both groups into a single DF
tp_ob = transitionProbabilities("ejer", ndata[ndata.Obesity == 0])
tp_obc = transitionProbabilities("ejer", ndata[ndata.Obesity == 1])
tp_ob["probsc"] = tp_obc.loc[:, "probs"]
tp_ob = tp_obc.rename(columns={"probs":"nobese", "probsc":"obese"})
tp_obc.to_csv(path_or_buf="ejer_obese_vs_nobese.csv", index=False)

In [112]:
# Non Higher Degree vs. Higher degree 
tp_deg = transitionProbabilities("ejer", ndata[ndata.hdegree == 0])
tp_degc = transitionProbabilities("ejer", ndata[ndata.hdegree == 1])
tp_deg["probsc"] = tp_degc.loc[:, "probs"]
tp_deg = tp_deg.rename(columns={"probs":"nhdegree", "probsc":"hdegree"})
tp_deg.to_csv(path_or_buf="ejer_nhdegree_vs_hdegree.csv", index=False)

In [111]:
tp_ob[tp_ob.pattern == "B->A"]

,labels,pattern,nobese
2,a_20: A | a_30: B,B->A,0.207317
6,a_10: A | a_30: B,B->A,0.317073
10,a_10: A | a_20: B,B->A,0.197917
14,a_5: A | a_30: B,B->A,0.341463
18,a_5: A | a_20: B,B->A,0.270833
22,a_5: A | a_10: B,B->A,0.166667
26,a_1: A | a_30: B,B->A,0.268293
30,a_1: A | a_20: B,B->A,0.208333
34,a_1: A | a_10: B,B->A,0.196078
38,a_1: A | a_5: B,B->A,0.112000
